In [ ]:
# !nvidia-smi

# 使用DeepSpeed进行全参数微调

## 安装依赖

In [ ]:
!git clone https://github.com/THUDM/ChatGLM2-6B.git

In [ ]:
!pip install -r ChatGLM2-6B/requirements.txt

In [ ]:
!pip install rouge_chinese nltk jieba datasets 

## 加载模型

In [ ]:
!git clone https://huggingface.co/THUDM/chatglm2-6b-int4

In [ ]:
# 加载模型
from transformers import AutoTokenizer, AutoModel

model_path = "chatglm2-6b-int4"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()
# model = model.eval()

In [ ]:
from IPython.display import display, Markdown, clear_output

# 准备提示语
prompt = "大模型需要具备的技能有哪些"

# 使用 IPython.display 流式打印模型输出
for response, history in model.stream_chat(
        tokenizer, prompt, history=[]):
    clear_output(wait=True)
    display(Markdown(response))

## 模型微调

In [ ]:
# 下载 ADGEN 数据集
!wget -O AdvertiseGen.tar.gz https://cloud.tsinghua.edu.cn/f/b3f119a008264b1cabd1/?dl=1

In [ ]:
# 解压数据集
!tar -xzvf AdvertiseGen.tar.gz

In [ ]:
!pip install deepspeed

In [ ]:
import os

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# !MASTER_PORT=$(shuf -n 1 -i 10000-65535)
# !LR=1e-4 && CUDA_VISIBLE_DEVICES=0,1 deepspeed --num_gpus=2 --master_port $MASTER_PORT  ChatGLM2-6B/ptuning/main.py \

# LR=1e-4 
!deepspeed --num_gpus=2 --master_port 520 ChatGLM2-6B/ptuning/main.py \
    --deepspeed ChatGLM2-6B/ptuning/deepspeed.json \
    --do_train \
    --train_file AdvertiseGen/train.json \
    --test_file AdvertiseGen/dev.json \
    --prompt_column content \
    --response_column summary \
    --overwrite_cache \
    --model_name_or_path chatglm2-6b-int4 \
    --output_dir ./output/adgen-chatglm2-6b-int4-pt \
    --overwrite_output_dir \
    --max_source_length 64 \
    --max_target_length 64 \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --predict_with_generate \
    --max_steps 100 \
    --logging_steps 10 \
    --save_steps 100 \
    --learning_rate 2e-2 \
    --fp16

## 模型部署-单卡

In [ ]:
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer

# Fine-tuning 后的表现测试，载入Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True, pre_seq_len=128)
model = AutoModel.from_pretrained(model_path, config=config, trust_remote_code=True)
# 此处使用你的 ptuning 工作目录
prefix_state_dict = torch.load(os.path.join("output/adgen-chatglm2-6b-int4-pt/checkpoint-100", "pytorch_model.bin"))
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    if k.startswith("transformer.prefix_encoder."):
        new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

# 根据需求可以进行量化，也可以直接使用：
# model = model.quantize(4)
model = model.half().cuda()
model.transformer.prefix_encoder.float()
model = model.eval()

## 模型部署（多卡部署）

In [ ]:
# import torch
# from transformers import AutoConfig, AutoModel, AutoTokenizer
# from utils import load_model_on_gpus

# # Fine-tuning 后的表现测试，载入Tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
# config = AutoConfig.from_pretrained(model_path, trust_remote_code=True, pre_seq_len=128)
# model = load_model_on_gpus(model_path, num_gpus=2)

# # 此处使用你的 ptuning 工作目录
# prefix_state_dict = torch.load(os.path.join("output/adgen-chatglm2-6b-int4-pt/checkpoint-100", "pytorch_model.bin"))
# new_prefix_state_dict = {}
# for k, v in prefix_state_dict.items():
#     if k.startswith("transformer.prefix_encoder."):
#         new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
# model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

# # 根据需求可以进行量化，也可以直接使用：
# # model = model.quantize(4)
# model = model.half().cuda()
# model.transformer.prefix_encoder.float()
# model = model.eval()

In [ ]:
# response, history = model.chat(tokenizer, "类型#上衣\*材质#牛仔布\*颜色#白色\*风格#简约\*图案#刺绣\*衣样式#外套\*衣款式#破洞", history=[])
# response